**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Tue Oct 12 08:42:19 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    26W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
"3'1 PPTS"
'3-2 PPTS'
 Abstract.pdf
'Android studio'
'ATCD Lab Project Titles.gsheet'
'Colab Notebooks'
 CVV.mp4
'DOSBOX Software'
'[FreeCourseSite.com] Udemy - Machine Learning A-Z™ Hands-On Python & R In Data Science'
'Getting started.pdf'
'Modern HTML & CSS From The Beginning (Including Sass)'
'My Drive'
 outputs
'Python by praveen'
'Resume (1).pdf'
 resume.pdf
 Resume.pdf
'Stock Price Prediction Using Time Series.pdf'
'vup-trfe-rso - Mar 15, 2021.gjam'
 yolov3


**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15316, done.
remote: Total 15316 (delta 0), reused 0 (delta 0), pack-reused 15316
Receiving objects: 100% (15316/15316), 13.71 MiB | 17.84 MiB/s, done.
Resolving deltas: 100% (10408/10408), done.


**2) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [ ]:
!echo "Hair" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

mkdir: cannot create directory ‘data/obj’: File exists


In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-10-12 08:46:47--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  56.0MB/s    in 2.8s    

2021-10-12 08:46:50 (56.0 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**3) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/bexar-arms-uLOWWVPVefQ-unsplash.jpg  
  inflating: data/obj/bexar-arms-uLOWWVPVefQ-unsplash.txt  
  inflating: data/obj/classes.txt    
  inflating: data/obj/jay-rembert-xnK_o6OdTew-unsplash.jpg  
  inflating: data/obj/jay-rembert-xnK_o6OdTew-unsplash.txt  
  inflating: data/obj/maxim-potkin-WFRBQ94Xhhc-unsplash.jpg  
  inflating: data/obj/maxim-potkin-WFRBQ94Xhhc-unsplash.txt  
  inflating: data/obj/roman-poberezhnik-41SKIzYcFh0-unsplash.jpg  
  inflating: data/obj/roman-poberezhnik-41SKIzYcFh0-unsplash.txt  
  inflating: data/obj/set-silhouettes-modern-assault-sniper-rifles-silhouettes-modern-assault-sniper-rifles-164477566.jpg  
  inflating: data/obj/set-silhouettes-modern-assault-sniper-rifles-silhouettes-modern-assault-sniper-rifles-164477566.txt  
  inflating: data/obj/steve-woods-cD2eM-TkE68-unsplash.jpg  
  inflating: data/obj/steve-woods-cD2eM-TkE68-unsplash.txt  
  inflating: data/obj/stngr-industries-s87cgU1YDgI-unsp

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

0 0.491250 0.455095 0.964167 0.768566

['0', '0.491250', '0.455095', '0.964167', '0.768566']
0 data/obj/AWM-338-white.txt
0 0.491250 0.455095 0.964167 0.768566
0 0.513451 0.329092 0.888315 0.482288

['0', '0.513451', '0.329092', '0.888315', '0.482288']
1 data/obj/alexander-andrews-rjIdMLBlFrY-unsplash.txt
0 0.513451 0.329092 0.888315 0.482288
0 0.498667 0.485417 0.888667 0.810000

['0', '0.498667', '0.485417', '0.888667', '0.810000']
2 data/obj/stngr-industries-s87cgU1YDgI-unsplash.txt
0 0.498667 0.485417 0.888667 0.810000
gun

[]
0 0.500868 0.573640 0.921875 0.406539

['0', '0.500868', '0.573640', '0.921875', '0.406539']
4 data/obj/thomas-def-MfM3p2yn4Ew-unsplash.txt
0 0.500868 0.573640 0.921875 0.406539
0 0.485725 0.405237 0.932099 0.548322

['0', '0.485725', '0.405237', '0.932099', '0.548322']
4 data/obj/thomas-def-MfM3p2yn4Ew-unsplash.txt
0 0.485725 0.405237 0.932099 0.548322
0 0.497708 0.500000 0.763750 0.793333

['0', '0.497708', '0.500000', '0.763750', '0.793333']
5 data/obj/bex

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/AWM-338-white.jpg', 'data/obj/jay-rembert-xnK_o6OdTew-unsplash.jpg', 'data/obj/roman-poberezhnik-41SKIzYcFh0-unsplash.jpg', 'data/obj/alexander-andrews-rjIdMLBlFrY-unsplash.jpg', 'data/obj/3ae48803c5c5e341e3bb57e27152f1f0.jpg', 'data/obj/stngr-industries-s87cgU1YDgI-unsplash.jpg', 'data/obj/steve-woods-cD2eM-TkE68-unsplash.jpg', 'data/obj/stngr-industries-YivlnxkYIDc-unsplash.jpg', 'data/obj/maxim-potkin-WFRBQ94Xhhc-unsplash.jpg', 'data/obj/thomas-def-MfM3p2yn4Ew-unsplash.jpg', 'data/obj/bexar-arms-uLOWWVPVefQ-unsplash.jpg', 'data/obj/set-silhouettes-modern-assault-sniper-rifles-silhouettes-modern-assault-sniper-rifles-164477566.jpg']


In [ ]:
#Create training.txt file
file = open("data/train.txt", "w")
file.write("\n".join(images_list))
file.close()

**4) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000569, iou_loss = 0.000000, total_loss = 0.000569 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000029, iou_loss = 0.000000, total_loss = 0.000029 
 total_bbox = 81312, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.526461), count: 5, class_loss = 0.808676, iou_loss = 0.775299, total_loss = 1.583974 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000097, iou_loss = 0.000000, total_loss = 0.000097 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000039, iou_loss = 0.000000, total_loss = 0.000039 
 total_bbox = 81317, rewritten_bbox = 0.000000 % 
v3 (mse